In [1]:
import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

In [2]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
#model = trt_pose.models.densenet121_baseline_att(num_parts, 2 * num_links).cuda().eval()

In [3]:
import torch

MODEL_WEIGHTS = 'resnet18_baseline_att_224x224_A_epoch_249.pth'
#MODEL_WEIGHTS = 'densenet121_baseline_att_256x256_B_epoch_160.pth'

model.load_state_dict(torch.load(MODEL_WEIGHTS))

<All keys matched successfully>

In [4]:
modelYolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/nathan/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv            

Adding autoShape... 


In [5]:
WIDTH = 224
HEIGHT = 224

#WIDTH = 256
#HEIGHT = 256

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

In [6]:
import time
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [7]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)
resize = transforms.Compose([transforms.ToPILImage(),transforms.Resize((HEIGHT, WIDTH))])

In [ ]:
vid = cv2.VideoCapture('HumanPose1.mp4')
vidRunning = True
    
while(vidRunning):
    tup, frame = vid.read()
    if tup:
        resize(frame)
        data = preprocess(frame)
        t0 = time.time()
        cmap, paf = model(data)
        t1 = time.time()
        
        cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
        counts, objects, peaks = parse_objects(cmap, paf)
        
        t2 = time.time()
        results = modelYolo(frame)
        t3 = time.time()
        
        quickMath = ((t1-t0) + (t3-t2))
        print("This took ", quickMath)
        
        draw_objects(frame, counts, objects, peaks)
        
        returned = results.render()
        finalImage = cv2.add(frame, returned[0])
        cv2.imshow('OUTPUT', finalImage)
        cv2.waitKey(1)
        
    else:
        vidRunning = False

This took  109.35719609260559
This took  12.527987480163574
This took  3.7289042472839355
This took  6.236683130264282
This took  18.82216453552246
This took  23.97421956062317
This took  4.771433591842651
This took  5.550743103027344
This took  4.3053998947143555
This took  6.556941986083984
This took  5.101637601852417
This took  4.723688125610352
This took  4.503425598144531
This took  7.05120587348938
This took  4.975395679473877
This took  9.608513116836548
This took  13.935269594192505
This took  6.488142967224121
This took  7.592902660369873
This took  10.196940183639526
This took  3.6204299926757812
This took  6.24502158164978
This took  6.269418954849243
This took  9.237916707992554
This took  7.572328567504883
This took  6.968639373779297
This took  8.68227243423462
This took  10.764209985733032
